<a href="https://colab.research.google.com/github/VivianOuou/NLP-Course/blob/main/course/en/chapter2/section4_Tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenizers (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [10]:
!pip install datasets evaluate transformers[sentencepiece]

### **基于单词的分词（Word-based Tokenization）方法总结**

#### **方法概述**
基于单词的分词（Word-based Tokenization）是最直观的分词方式，通常按照**空格和标点符号**将文本拆分成单词，并为每个单词分配唯一的ID。例如：
```python
text = "Jim Henson was a puppeteer"
tokens = text.split()  # ['Jim', 'Henson', 'was', 'a', 'puppeteer']
```

#### **优点**
✅ **简单易用**  
- 实现容易，仅需按空格或标点分割文本，适合基础NLP任务。  
- 人类可读性强，分词结果直接对应自然语言单词。

✅ **语义粒度较细**  
- 每个单词独立编码，适合处理固定词汇表的任务（如分类、检索）。  

✅ **适用于小规模、领域特定的数据**  
- 如果词汇表可控（如医学、法律术语），单词级分词可能足够高效。

---

#### **缺点**
❌ **词汇表膨胀（Vocabulary Bloat）**  
- 英语约有50万+单词，若考虑变形（如"run"→"running"），词汇量会极大。  
- 存储和计算成本高，模型输入层需要极大矩阵（如 `500,000×768` 的嵌入矩阵）。

❌ **无法处理未登录词（OOV, Out-of-Vocabulary）**  
- 遇到不在词汇表的单词（如专业术语、拼写错误）会被替换为 `[UNK]`，导致信息丢失。  
- 例如：
  ```python
  vocab = {"cat": 1, "dog": 2, "[UNK]": 0}
  encode("puppy") → [0]  # 完全丢失语义
  ```

❌ **无法捕捉词形变化（Morphology）**  
- 将不同形式的单词视为完全无关：  
  - "dog" 和 "dogs" → 不同ID  
  - "run" 和 "ran" → 无关联  

❌ **对空格和标点敏感**  
- 某些语言（如中文、日语）没有明确空格分隔，难以直接应用。  
- 缩写和连字符可能被错误分割（如 "can't" → `["can", "'", "t"]`）。

---


**建议**：除非处理特定领域数据，否则优先选择基于子词的分词方法（如Hugging Face的`BertTokenizer`）。

### **基于字符的分词（Character-based Tokenization）方法总结**

#### **方法概述**
基于字符的分词（Character-based Tokenization）将文本拆分为**单个字符**（字母、标点、空格），而不是单词。例如：
```python
text = "dog"
tokens = list(text)  # ['d', 'o', 'g']
```

#### **优点**
✅ **极小的词汇量**  
- 仅需覆盖语言的基本字符集（如ASCII共256个字符，中文约5000+常用字）。  
- 词汇量通常 **< 1,000**，远小于单词级分词（50万+）。

✅ **几乎无OOV（未登录词）问题**  
- 任何单词均可由字符组合而成，避免 `[UNK]` 问题。  
  ```python
  # 即使词汇表没有"puppy"，仍可拆解为：
  list("puppy") → ['p', 'u', 'p', 'p', 'y']
  ```

✅ **跨语言兼容性强**  
- 适合无空格语言（如中文、日语）：  
  ```python
  list("深度学习") → ['深', '度', '学', '习']
  ```
- 对拼写错误、缩写更鲁棒（如 "can't" → `['c', 'a', 'n', "'", 't']`）。

---

#### **缺点**
❌ **序列长度爆炸**  
- 一个单词变为多个字符，输入序列大幅变长：  
  ```python
  "Transformers" → 12个字符（原本1个单词）
  ```
  - 导致计算量增加（Transformer的复杂度与序列长度平方相关）。

❌ **语义粒度太细**  
- 单个字符无明确语义（如 'd' 与 'o' 组合才形成 "do" 的含义）。  
- 模型需额外学习字符组合的语义，训练难度更高。

❌ **对空格和标点敏感**  
- 空格可能被当作普通字符处理，需额外规则：  
  ```python
  "hello world" → ['h', 'e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd']
  ```

❌ **信息密度低**  
- 拉丁语系中，单个字符信息量少（中文/日文稍好）：  
  - 英文：'a' 单独无意义  
  - 中文：'语' 本身有部分语义  


### **子词分词（Subword Tokenization）方法总结**

#### **核心思想**
子词分词（Subword Tokenization）是一种**平衡词汇量与语义表达**的分词方法，其核心原则是：
- **高频词保留完整**（如 "the", "cat"）  
- **低频词拆分为有意义的子词**（如 "annoyingly" → `"annoying" + "ly"`）  

#### **主流算法对比**
| **算法**         | **代表模型** | **特点**                                                                 | **示例**                          |
|------------------|--------------|--------------------------------------------------------------------------|-----------------------------------|
| **Byte-Pair Encoding (BPE)** | GPT-2, GPT-3 | 通过迭代合并最高频的字符对构建词汇表                                    | "tokenization" → `["token", "ization"]` |
| **WordPiece**    | BERT         | 类似BPE，但基于概率合并（最大化语言模型似然）                           | "unhappiness" → `["un", "##happiness"]` |
| **Unigram LM**   | XLNet, ALBERT| 从大词汇表开始，逐步删除低概率子词                                      | "深度学习" → `["深", "度", "学习"]`     |
| **SentencePiece**| T5, mBERT    | 直接处理原始文本（无需预分词），支持多语言                              | "Hello world!" → `["▁He", "llo", "▁world", "!"]` |

---

### **三大优势**
1. **词汇量可控**  
   - 典型词汇量：30,000–50,000（远小于单词级分词的50万+）  
   - 例如：BERT的`bert-base-uncased`词汇量=30,522  

2. **近乎零OOV问题**  
   - 任何单词均可拆解为子词：  
     ```python
     # 即使词汇表没有"tokenization"
     "tokenization" → "token" + "ization"  # 两个已知子词
     ```

3. **保留语义组合性**  
   - 子词本身携带语义：  
     - `"##ly"` 表示副词（如 "quickly" → `"quick" + "##ly"`）  
     - `"##ization"` 表示名词化（如 "modernization" → `"modern" + "##ization"`）  

---

### **实际应用示例**
#### **1. 使用Hugging Face Tokenizer**
```python
from transformers import AutoTokenizer

# 加载BERT的WordPiece分词器
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer.tokenize("unhappiness")
print(tokens)  # ['un', '##ha', '##pp', '##iness']
```

#### **2. 处理多语言（SentencePiece）**
```python
from transformers import AutoTokenizer

# 加载T5的SentencePiece分词器
tokenizer = AutoTokenizer.from_pretrained("t5-base")
tokens = tokenizer.tokenize("こんにちは世界！")  # 日语"你好世界！"
print(tokens)  # ['▁', 'こん', 'にち', 'は', '世界', '!']
```

---

### **为何优于字符/单词级分词？**
| **场景**               | **单词级**               | **字符级**               | **子词级**              |
|------------------------|--------------------------|--------------------------|-------------------------|
| 词汇量                 | 极大（50万+）            | 极小（<1,000）           | 中等（3万–5万）         |
| OOV问题                | 严重                     | 无                       | 几乎无                  |
| 语义粒度               | 过粗（无法处理词形变化） | 过细（字符无独立语义）   | 适中（保留语义单元）    |
| 序列长度               | 短                       | 极长                     | 中等                    |
| 适合任务               | 固定词汇表任务           | 语音/拼写错误处理        | 通用NLP任务             |

---

### **语言适应性**
1. **黏着语（Agglutinative Languages）**  
   - 如土耳其语、芬兰语：  
     ```python
     # 土耳其语"öğretmenlerimizden"（来自我们的老师们）
     → ["öğret", "##men", "##ler", "##imiz", "##den"]
     ```
2. **屈折语（Fusional Languages）**  
   - 如英语、德语：  
     ```python
     "running" → ["run", "##n", "##ing"]
     ```
3. **孤立语（Isolating Languages）**  
   - 如汉语：  
     ```python
     "深度学习" → ["深", "度", "学习"]
     ```

---

### **进阶话题**
1. **混合分词策略**  
   - 对常见词保留完整，罕见词回退到字符级：
     ```python
     "ChatGPT" → ["Chat", "G", "PT"]  # GPT-4的处理方式
     ```
2. **跨语言迁移**  
   - 使用SentencePiece训练统一的多语言分词器（如mBERT支持100+语言）。

3. **领域自适应**  
   - 在专业领域（生物、法律）上继续训练分词器：
     ```python
     # 添加生物医学术语到词汇表
     tokenizer.add_tokens(["EGFR", "CRISPR-Cas9"])
     ```

---

### **总结**
- **推荐使用子词分词**：在绝大多数NLP任务中（如文本分类、翻译、生成），子词分词提供了最佳平衡。  
- **工具推荐**：Hugging Face的`transformers`库支持所有主流子词算法（WordPiece/BPE/SentencePiece）。  
- **实践建议**：直接使用预训练模型的分词器，而非从头训练。  

通过子词分词，我们能够以**紧凑的词汇表**覆盖**近乎无限的单词表达**，这正是现代Transformer模型高效处理多语言、多领域文本的核心基础。

In [11]:
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)

['Jim', 'Henson', 'was', 'a', 'puppeteer']


分词器（Tokenizer）的加载与保存方法

核心方法

与加载/保存模型类似，分词器也使用以下两个关键方法：

from_pretrained()：从Hugging Face Hub或本地加载分词器
save_pretrained()：保存分词器到本地目录

In [12]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [14]:
tokenizer("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer.save_pretrained("directory_on_my_computer")

('directory_on_my_computer/tokenizer_config.json',
 'directory_on_my_computer/special_tokens_map.json',
 'directory_on_my_computer/vocab.txt',
 'directory_on_my_computer/added_tokens.json',
 'directory_on_my_computer/tokenizer.json')

Encoding

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


In [17]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


In [20]:
#作业练习题1
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
text1 = "I've been waiting for a HuggingFace course my whole life."
tokens1 = tokenizer.tokenize(text1)
print(tokens1)

['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.']


In [21]:
ids1 = tokenizer.convert_tokens_to_ids(tokens1)
print(ids1)

[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


In [22]:
#作业练习题2

text2 = "I hate this so much!"
tokens2 = tokenizer.tokenize(text2)
print(tokens2)

ids2 = tokenizer.convert_tokens_to_ids(tokens2)
print(ids2)

['i', 'hate', 'this', 'so', 'much', '!']
[1045, 5223, 2023, 2061, 2172, 999]


**加粗文字**### **解码（Decoding）过程详解**

#### **1. 核心功能**
解码是编码的逆过程，将模型输出的数字ID序列转换回可读文本。通过`decode()`方法实现：
```python
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
input_ids = [7993, 170, 11303, 1200, 2443, 1110, 3014]

decoded_text = tokenizer.decode(input_ids)
print(decoded_text)
```
**输出**：
```
"Using a Transformer network is simple"
```

---

#### **2. 关键技术细节**
| **特性**                | **说明**                                                                 | **示例**                          |
|-------------------------|--------------------------------------------------------------------------|-----------------------------------|
| **子词重组**            | 自动合并`##`开头的子词                                                  | `["Transform", "##er"]` → "Transformer" |
| **特殊token过滤**       | 默认跳过[CLS]、[SEP]等特殊标记                                          | `[101, 7993, 102]` → "Using"     |
| **空格恢复**            | 根据语言规则智能添加空格                                                 | `["Hello", "world"]` → "Hello world" |
| **标点处理**            | 自动处理标点粘连问题                                                     | `["hello", "!"]]` → "hello!"     |

---

#### **3. 完整工作流程**
```mermaid
graph LR
    A[Input IDs] --> B(Tokenizer.decode)
    B --> C[合并子词]
    B --> D[过滤特殊token]
    B --> E[调整空格]
    B --> F[输出可读文本]
```

---

#### **4. 高级用法示例**
**场景1：处理模型生成结果**
```python
# 假设模型生成输出（带特殊token）
generated_ids = [101, 7993, 170, 11303, 1200, 102]  # [CLS] Using a Transformer [SEP]

print(tokenizer.decode(generated_ids))  # 输出: "Using a Transformer"
print(tokenizer.decode(generated_ids, skip_special_tokens=False))  # 输出: "[CLS] Using a Transformer [SEP]"
```

**场景2：控制空格输出**
```python
# 中文等无空格语言处理
chinese_ids = tokenizer.encode("深度学习").input_ids  # [101, 123, 456, 789, 102]
print(tokenizer.decode(chinese_ids))  # 输出: "深度学习"
```

**场景3：修复子词断字**
```python
# 修复子词导致的断字问题
broken_ids = [17662, 12172]  # ["hugging", "##face"]
print(tokenizer.decode(broken_ids))  # 输出: "HuggingFace"
```

---

#### **5. 为什么解码比简单拼接复杂？**
假设直接拼接分词结果：
```python
tokens = ["Using", "a", "Transform", "##er"]
bad_text = " ".join(tokens)  # 得到: "Using a Transform ##er"（错误！）
```
**解码器实际做了**：
1. 移除`##`标记 → `"er"`
2. 合并相邻子词 → `"Transformer"`
3. 智能空格处理 → `"a Transformer"`

---



In [18]:
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

Using a transformer network is simple
